<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/Treinamento_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LSTM MEI CHUCHU

In [ ]:
from sklearn import metrics
from keras import layers, models, optimizers

def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(classifier.summary())
    classifier.fit(feature_vector_train, label, epochs=3)
    return classifier

def create_rnn_lstm(embedding_matrix):
  
  model_input = layers.Input(shape=(50, ),name="input",dtype='int32')
  embedding = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix])(model_input)
  embedding = layers.SpatialDropout1D(0.3)(embedding)
  lstm = layers.LSTM(1000, dropout =0.1, name="LSTM")(embedding)
  output = layers.Dense(1, activation='sigmoid',name='sigmoid')(lstm)
  model = models.Model(inputs=model_input,outputs=output)
  return model

classifier = create_rnn_lstm(embedding_matrix)
trainedModel = train_model(classifier, train_seq_x, train_y, valid_seq_x)
# predict the labels on validation dataset
predictions = trainedModel.predict(valid_seq_x)
predictions = predictions.round()

print("RNN-LSTM, Word Embeddings",  metrics.accuracy_score(valid_y,predictions))
print(predictions)